In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [192]:
# Neural Network with hidden units
def normalize(x):
    mean = np.mean(x)
    std_dev = np.std(x)
    z = (x - mean) / std_dev
    return z

def sigmoid(z):
    z_clipped = np.clip(z, -500, 500)  # Clip values to the range [-500, 500]
    return 1 / (1 + np.exp(-z_clipped))

def sigmoid_derivative(z):
    gz = sigmoid(z)
    return gz * (1 - gz)

def linear(x,w,b):
    return (w * x) + b

def relu(z):
    return np.maximum(0,z)

def relu_derivative(z):
     return np.where(z > 0, 1, 0)

def tanh_derivative(z):
    return 1 - (tanh(z) ** 2)

def leaky_relu(z):
    return np.maximum(0.01 * z, z)

def tanh(z):
    return np.tanh(z)

def loss_sigmoid(y_pred, y):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.mean(- (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))) 
    return loss

def init_params(x, hidden_units, output_units):
    input_units = x.shape[1]
    w1 = np.random.randn(hidden_units, input_units)
    b1 = np.zeros((hidden_units,  output_units))
    w2 = np.random.randn(output_units, hidden_units)
    b2 = np.zeros((output_units, 1))
    
    return w1, b1, w2, b2
    
def back_prop(x,y,a1,a2,w2,z1):
    m = x.shape[1]
    
    dz2 = a2 - y # y stacked horizontally
    dw2 = np.dot(dz2 ,a1.T) / m
    db2 = (np.sum(dz2, axis=1, keepdims=True)) / m
    dz1 = np.dot(w2.T,dz2) * relu_derivative(z1)
    dw1 = np.dot(dz1 ,x) / m
    db1 = (np.sum(dz1, axis=1, keepdims=True)) / m
    
    return dw1, db1, dw2, db2

def forward_prop(x, y, hidden_units, learning_rate, epochs):
    w1, b1, w2, b2 = init_params(x, hidden_units, output_units=1)
    
    for i in range(epochs+1):
        z1 = np.dot(w1,x.T) + b1
        a1 = relu(z1)
        z2 = np.dot(w2, a1) + b2
        a2 = sigmoid(z2)
        
        dw1, db1, dw2, db2 = back_prop(x, y, a1, a2, w2, z1)
        
        w1 -= learning_rate * dw1
        b1 -= learning_rate * db1
        w2 -= learning_rate * dw2 
        b2 -= learning_rate * db2
        
        if i % math.ceil(epochs/10) == 0:
            print(f"Iteration: {i} Loss: {loss_sigmoid(a2, y)}")
    
    return w1, b1, w2, b2

def predict(x, w1, b1, w2, b2):
    z1 = np.dot(x, w1.T) + b1.reshape(1,w1.shape[0])
    a1 = relu(z1)
    z2 = np.dot(a1, w2.T) + b2
    a2 = sigmoid(z2)
    
    y_hat = (a2 > 0.5).astype(int)
    
    return y_hat.T

def accuracy(yhat, y):
    return  (np.sum(yhat == y) / y.shape[0]) * 100
    
fit = forward_prop

In [187]:
# Data preprocessing
data = pd.read_csv('breast-cancer.csv')
# Labelling
mapping = {'M': 1, 'B' : 0}
data.diagnosis = data.diagnosis.map(mapping)
# Dropping
data.drop('id', axis=1, inplace=True)

In [188]:
# Train/Test set
x = data.drop('diagnosis', axis=1)
y = data['diagnosis']

x_train = x[:400].values
x_test = x[400:].values
y_train = y[:400].values
y_test = y[400:].values

x_train_nz = normalize(x_train)
x_test_nz = normalize(x_test)

In [193]:
w1, b1, w2, b2 = fit(x_train_nz, y_train, hidden_units=64, learning_rate=0.01, epochs=3000)

Iteration: 0 Loss: 2.938586775479914
Iteration: 300 Loss: 0.15418491658776518
Iteration: 600 Loss: 0.144694733454086
Iteration: 900 Loss: 0.13928000019847417
Iteration: 1200 Loss: 0.1349091200681611
Iteration: 1500 Loss: 0.13015208551625623
Iteration: 1800 Loss: 0.12384577701967402
Iteration: 2100 Loss: 0.12994502828878585
Iteration: 2400 Loss: 0.1276853322865138
Iteration: 2700 Loss: 0.12580614992345832
Iteration: 3000 Loss: 0.12419370301919114


In [194]:
yhat = predict(x_train_nz,w1,b1,w2,b2)
accuracy(yhat, y_train)

94.5

In [195]:
yhat_test = predict(x_test_nz,w1,b1,w2,b2)
accuracy(yhat_test, y_test)

88.75739644970415